In [67]:
from networkx.algorithms.community.centrality import girvan_newman
from networkx.algorithms import community
import networkx as nx
import numpy as np
import community.community_louvain as community_louvain
import matplotlib.pyplot as plt
from numpy import genfromtxt
import os
from numpy import savetxt
import time
plt.rcParams['figure.figsize'] = 15,10

# Datasets

In [24]:
def read_datasets(path):
    directory = os.fsencode(path)
    datasets = []
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        file_path = os.path.join(path, filename)
        parts = filename.split("=")
        if len(parts) > 1:
            k = int("".join(filter(str.isdigit, parts[1])))
            datasets.append([file_path, k]) 
        else:
            datasets.append([file_path])  
    return datasets

In [25]:
path = 'datasets'
datasets = read_datasets(os.path.join(os.getcwd(), path))
print(datasets)

[['C:\\Users\\Monika\\Desktop\\Studia\\SN\\Untitled Folder\\competition\\datasets\\D1-K=2.csv', 2], ['C:\\Users\\Monika\\Desktop\\Studia\\SN\\Untitled Folder\\competition\\datasets\\D1-UNC.csv'], ['C:\\Users\\Monika\\Desktop\\Studia\\SN\\Untitled Folder\\competition\\datasets\\D2-K=7.csv', 7], ['C:\\Users\\Monika\\Desktop\\Studia\\SN\\Untitled Folder\\competition\\datasets\\D2-UNC.csv'], ['C:\\Users\\Monika\\Desktop\\Studia\\SN\\Untitled Folder\\competition\\datasets\\D3-K=12.csv', 12], ['C:\\Users\\Monika\\Desktop\\Studia\\SN\\Untitled Folder\\competition\\datasets\\D3-UNC.csv']]


# Algorithms

In [68]:
def edge_to_remove(G):
    scores = nx.edge_betweenness_centrality(G)
    scores = dict(sorted(scores.items(), key=lambda item: item[1], reverse = True))
    return list(scores.keys())[0]

def create_list(cc):
    partition = []
    for c in cc:
        partition.append(c)
    return partition
    
def gn(G, k=None):
    cc = nx.connected_components(G)
    cc_count = nx.number_connected_components(G)
    n = G.number_of_nodes()
    scores_dict = {}
    if k == None:
        stop = n-1
    else:
        stop = k+1

    while(cc_count < stop):
        vs = edge_to_remove(G)
        G.remove_edge(vs[0], vs[1])
        cc = nx.connected_components(G)
        new_cc_count = nx.number_connected_components(G)
        if new_cc_count > cc_count:
            part_list = create_list(cc)
            score = community.quality.modularity(G, part_list)
            scores_dict[new_cc_count] = [score, part_list]
            cc_count = new_cc_count
    scores_dict = dict(sorted(scores_dict.items(), key=lambda item: item[1][0], reverse = True))
    scores = scores_dict.items()
    if k == None:
        score = list(scores)[0]
    else:
        score = scores_dict[k]

In [58]:
def edge_to_remove(G):
    scores = nx.edge_betweenness_centrality(G)
    scores = dict(sorted(scores.items(), key=lambda item: item[1], reverse = True))
    return list(scores.keys())[0]

def create_list(cc):
    partition = []
    for c in cc:
        partition.append(c)
    return partition

def prepare_results(score):
    clusters = score[1]
    results = []
    for i, cluster in enumerate(clusters):
        for point in cluster:
            results.append([point+1, i+1])
    results = np.array(results).astype(int)
    ind = np.argsort(results[:, 0])
    results = results[ind]
    return np.array(results).astype(int)
        
def gn(G, k=None):
    cc = nx.connected_components(G)
    cc_count = nx.number_connected_components(G)
    n = G.number_of_nodes()
    scores_dict = {}
    if k == None:
        stop = n-1
    else:
        stop = k+1

    while(cc_count < stop):
        vs = edge_to_remove(G)
        G.remove_edge(vs[0], vs[1])
        cc = nx.connected_components(G)
        new_cc_count = nx.number_connected_components(G)
        if new_cc_count > cc_count:
            part_list = create_list(cc)
            score = community.quality.modularity(G, part_list)
            scores_dict[new_cc_count] = [score, part_list]
            cc_count = new_cc_count
    scores_dict = dict(sorted(scores_dict.items(), key=lambda item: item[1][0], reverse = True))
    scores = scores_dict.values()
    if k == None:
        score = list(scores)[0]
    else:
        score = scores_dict[k]
    return score

In [65]:
def calc_results(datasets, path_results=None):
    for dataset in datasets:
        if len(dataset) > 1:
            path, k = dataset
        else:
            path = dataset[0]
            k = None
        
        with open(path, 'r') as f:
            my_data = genfromtxt(f, delimiter=',')
        f.close()
        
        G = nx.from_numpy_matrix(my_data)
        start_time = time.time()
        scores = gn(G.copy(), k)
        file_name = os.path.basename(path)
        print(f'Time for {file_name}: {time.time() - start_time} s')
        scores = prepare_results(scores)
        path_out = os.path.join(path_results, file_name)
        savetxt(path_out, scores, delimiter=',', fmt='%i')       

# Results

In [66]:
calc_results(datasets, path_results='results')

Time for D1-K=2.csv: 0.0498652458190918 s
Time for D1-UNC.csv: 2.7904694080352783 s
Time for D2-K=7.csv: 0.26976966857910156 s
Time for D2-UNC.csv: 17.86822509765625 s
Time for D3-K=12.csv: 5.585643291473389 s
Time for D3-UNC.csv: 3.284228801727295 s
